# YOLO for both Satellite and for Drone

In [3]:
import pandas as pd
from PIL import Image

# Load the .pkl file
df = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Train.pkl")
#df = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Val.pkl")

Preprocessing for YOLO

In [5]:

def scale_polygon(polygon, scale=1.15):
    """Scales a polygon outward from its centroid."""
    xs, ys = zip(*polygon)
    cx, cy = sum(xs) / len(xs), sum(ys) / len(ys)
    return [(cx + scale * (x - cx), cy + scale * (y - cy)) for x, y in polygon]

def polygon_to_yolo_bbox(polygon, img_width, img_height):
    """Converts polygon to YOLO normalized bbox."""
    xs, ys = zip(*polygon)
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    
    x_center = ((x_min + x_max) / 2) / img_width
    y_center = ((y_min + y_max) / 2) / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    
    return x_center, y_center, width, height

In [6]:
import os

#label_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\labels\val"  # or val, depending on the split
label_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\labels\train"  # or val, depending on the split
image_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\images"
os.makedirs(label_dir, exist_ok=True)

image_width, image_height = 600, 600  # Replace with actual size if dynamic

for idx, row in df.iterrows():
    label_lines = []

    # Process boilers (class 0)
    for poly in row['polygons_boil']:
        scaled_poly = scale_polygon(poly, scale=1.15)
        x, y, w, h = polygon_to_yolo_bbox(scaled_poly, image_width, image_height)
        label_lines.append(f"0 {x:.6f} {y:.6f} {w:.6f} {h:.6f}")

    # Process panels (class 1)
    for poly in row['polygons_pan']:
        scaled_poly = scale_polygon(poly, scale=1.15)
        x, y, w, h = polygon_to_yolo_bbox(scaled_poly, image_width, image_height)
        label_lines.append(f"1 {x:.6f} {y:.6f} {w:.6f} {h:.6f}")

    # Save to .txt file
    label_path = os.path.join(label_dir, f"{row['img_id']}.txt")
    with open(label_path, "w") as f:
        f.write("\n".join(label_lines))

In [ ]:
import os
import shutil
import pandas as pd

# Load the training DataFrame
df_train = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Train.pkl")
df_val = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Val.pkl")
# Paths
image_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\images"  # Folder with original images
output_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\val"  # Destination folder
os.makedirs(output_dir, exist_ok=True)

# Copy each image
for img_id in df_val['img_id']:
    src_path = os.path.join(image_dir, f"{img_id}.jpg")
    dst_path = os.path.join(output_dir, f"{img_id}.jpg")

    if os.path.exists(src_path):
        shutil.copy2(src_path, dst_path)
    else:
        print(f"⚠️ Warning: Image not found: {src_path}")


Training YOLO

In [ ]:
import os

label_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\labels\train"
image_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\train"

label_files = set(f for f in os.listdir(label_dir) if f.endswith(".txt"))
image_files = set(f for f in os.listdir(image_dir) if f.endswith(".jpg"))

# Check for orphaned files (label without image, image without label)
for f in label_files.union(image_files):
    label_path = os.path.join(label_dir, f.replace(".jpg", ".txt"))
    image_path = os.path.join(image_dir, f.replace(".txt", ".jpg"))

    label_exists = os.path.exists(label_path)
    image_exists = os.path.exists(image_path)

    if not label_exists or not image_exists:
        if label_exists:
            os.remove(label_path)
            print(f"🗑️ Removed orphan label: {label_path}")
        if image_exists:
            os.remove(image_path)
            print(f"🗑️ Removed orphan image: {image_path}")

# Now validate all remaining label files
for file in os.listdir(label_dir):
    if not file.endswith(".txt"):
        continue

    label_path = os.path.join(label_dir, file)
    image_path = os.path.join(image_dir, file.replace(".txt", ".jpg"))

    with open(label_path, "r") as f:
        lines = f.readlines()

    valid_lines = []
    for line in lines:
        try:
            parts = list(map(float, line.strip().split()))
            if (
                len(parts) == 5
                and 0 <= parts[1] <= 1 and 0 <= parts[2] <= 1
                and 0 < parts[3] <= 1 and 0 < parts[4] <= 1
            ):
                valid_lines.append(line)
        except:
            continue

    if valid_lines:
        with open(label_path, "w") as f:
            f.writelines(valid_lines)
    else:
        os.remove(label_path)
        if os.path.exists(image_path):
            os.remove(image_path)
            print(f"🧹 Removed image+label with no valid annotations: {file.replace('.txt', '')}")


In [ ]:
for file in os.listdir(label_dir):
    filepath = os.path.join(label_dir, file)
    with open(filepath, "r") as f:
        lines = f.readlines()
    valid_lines = []
    for line in lines:
        try:
            parts = list(map(float, line.strip().split()))
            if 0 <= parts[1] <= 1 and 0 <= parts[2] <= 1 and 0 < parts[3] <= 1 and 0 < parts[4] <= 1:
                valid_lines.append(line)
        except:
            continue
    if valid_lines:
        with open(filepath, "w") as f:
            f.writelines(valid_lines)
    else:
        os.remove(filepath)  # Delete file with no valid labels


In [ ]:
import torch
import gc

# Clear cache and free memory
torch.cuda.empty_cache()
gc.collect()


0

In [ ]:
from ultralytics import YOLO
import yaml

# Load your pre-trained YOLOv8 model
model = YOLO("yolov8s.pt")

# ✅ Load hyperparameters manually from hyp.yaml
with open(r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\hyp.yaml", "r") as f:
    hyp_params = yaml.safe_load(f)  # Converts YAML to dictionary

# ✅ Pass hyperparameters as individual keyword arguments
model.train(
    data=r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\yolo.yaml",  # Dataset config
    epochs=100,      # Total training epochs
    imgsz=512,       # Image size (try 768, 1024 for better accuracy)
    batch=16,        # Batch size (increase if GPU allows)
    amp=True,        # Automatic Mixed Precision (for speedup)
    conf=0.2,        # Confidence threshold for predictions
    iou=0.6,         # Intersection Over Union threshold
    optimizer="SGD", # Optimizer choice
    device=0,        # Use GPU 0
    workers=4,       # Number of data workers (depends on CPU)
    cache=True,      # Cache images to RAM for faster training
    val=True,        # Run validation during training
    verbose=True,    # Print logs during training
    save=True,       # Save best and final models
    name="train_safe_v2",  # Custom name for experiment folder
    **hyp_params  # ✅ Pass all hyperparameters from hyp.yaml
)


Ultralytics 8.3.94  Python-3.10.16 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\gnvca\OneDrive\Desktop\JP\model03S\yolo.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=True, device=0, workers=4, project=None, name=train_safe_v25, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.2, iou=0.6, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

train: Scanning C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\labels\train... 2439 images, 0 backgrounds, 1 corrupt: 100%|██████████| 2439/2439 [00:01<00:00, 1246.93it/s]

train: WARNING  C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\train\IDcxTl9zDOWTNH.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\train\IDpNecgm1Opkw.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.155635 2.054698]


train: New cache created: C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\labels\train.cache
WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (1.3GB RAM): 100%|██████████| 2438/2438 [00:20<00:00, 120.17it/s]
val: Scanning C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\labels\val.cache... 610 images, 0 backgrounds, 0 corrupt: 100%|██████████| 610/610 [00:00<?, ?it/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     64/100      3.35G     0.9952      1.301     0.9703         14        512: 100%|██████████| 153/153 [00:20<00:00,  7.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:02<00:00,  7.49it/s]

                   all        610       1153      0.542       0.25      0.365      0.152



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     65/100      3.35G     0.9569      1.263     0.9592         93        512:  54%|█████▎    | 82/153 [00:11<00:09,  7.41it/s]

In [8]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("runs/detect/train_safe4/weights/best.pt")

# Export to ONNX
model.export(format="onnx")

Ultralytics 8.3.94  Python-3.10.16 torch-2.6.0+cu126 CPU (AMD Ryzen 5 7600 6-Core Processor)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\train_safe4\weights\best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 6, 3549) (5.9 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ---------------------------------------- 14.5/14.5 MB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 213.1/213.1 MB 2.3 MB/s eta 0:00:00

requirements: AutoUpdate success  285.9s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu']
requirements:  Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success  288.0s, saved as 'runs\detect\train_safe4\weights\best.onnx' (11.6 MB)

Export complete (290.0s)


'runs\\detect\\train_safe4\\weights\\best.onnx'

In [26]:
from ultralytics import YOLO
import pandas as pd
import os

# Load trained model
model = YOLO("runs/detect/train_safe4/weights/best.pt")

# Run inference on test images
results = list(model.predict(source=r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test", save=False))

# Debug: Check if results exist
print("Total images processed:", len(results))
if len(results) == 0:
    print("⚠️ No images processed! Check your dataset path.")
    



image 1/610 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID014O6EC7.jpg: 320x416 2 panels, 8.5ms
image 2/610 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID06Zb8HWf7XPxodI.jpg: 320x416 2 panels, 9.8ms
image 3/610 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID0BqUUCBEQD.jpg: 320x416 2 panels, 13.0ms
image 4/610 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID0GSCMR.jpg: 320x416 (no detections), 14.4ms
image 5/610 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID0K20RbBcx5oQTp.jpg: 320x416 1 panel, 16.0ms
image 6/610 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID0LlEkllD.jpg: 320x416 (no detections), 10.0ms
image 7/610 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID0XksrqV55J4rH.jpg: 320x416 (no detections), 7.8ms
image 8/610 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID0Y4qEEE8Hqeap.jpg: 320x416 2 panels, 8.4ms
image 9/610 C:\Users\gnv

In [ ]:
# Store results
rows = []

for result in results:
    image_id = os.path.splitext(os.path.basename(result.path))[0]  # Get filename without extension
    class_ids = result.boxes.cls.int().tolist()
    
    boiler_count = class_ids.count(0)  # assuming 'boiler' = class 0
    panel_count = class_ids.count(1)   # assuming 'panel' = class 1
    
    # Append each class as a separate row
    rows.append({"ID": f"{image_id}_boil", "Target": boiler_count})
    rows.append({"ID": f"{image_id}_pan", "Target": panel_count})

# Debug: Check if rows are being created
print("First 5 rows:", rows[:5])

# Create DataFrame and save CSV
df = pd.DataFrame(rows)

# If dataframe is empty, print a warning
if df.empty:
    print("⚠️ WARNING: DataFrame is empty! Check your detection logic.")

df.to_csv("test_detection_counts.csv", index=False)
print("✅ CSV saved as 'test_detection_counts.csv'")


In [1]:
from ultralytics import YOLO
import pandas as pd
import os
import glob

# Load trained model
model = YOLO("runs/detect/train_safe4/weights/best.pt")

# Get all test images (ensure they're found)
image_files = glob.glob(r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\*")  # Any image format
if not image_files:
    raise ValueError("⚠️ No test images found! Check dataset path or file extensions.")

print(f"✅ Found {len(image_files)} test images.")

# Run inference on the test folder
results = model.predict(source=r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\\", save=False)

# Process results
rows = []
processed_images = set()

for result in results:
    image_id = os.path.splitext(os.path.basename(result.path))[0]
    processed_images.add(image_id)

    class_ids = result.boxes.cls.int().tolist()
    boiler_count = class_ids.count(0)
    panel_count = class_ids.count(1)

    rows.append({"ID": f"{image_id}_boil", "Target": boiler_count})
    rows.append({"ID": f"{image_id}_pan", "Target": panel_count})

# Ensure missing images have Target=0
for img_path in image_files:
    img_id = os.path.splitext(os.path.basename(img_path))[0]
    if img_id not in processed_images:
        rows.append({"ID": f"{img_id}_boil", "Target": 0})
        rows.append({"ID": f"{img_id}_pan", "Target": 0})

# Save CSV
df = pd.DataFrame(rows)
df.to_csv("test_detection_counts.csv", index=False)
print("✅ CSV saved: 'test_detection_counts.csv'")

✅ Found 1107 test images.


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID00qprY.jpg: 320x416 2 panels, 30.4ms
image 2/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID01AciUc.jpg: 320x416 2 panels, 5.4ms
image 3/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID0328D.jpg: 320x416 1 panel, 5.8ms
image 4/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID05WxObCF